In [2]:
import numpy as np
import onnx
import onnx.numpy_helper as np_helper
from onnx import TensorProto, helper
from pkgutil import get_data

import qonnx.core.onnx_exec as oxe
from qonnx.core.datatype import DataType
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.merge_onnx_models import MergeONNXModels

In [19]:
# raw_m = get_data("qonnx.data", "onnx/mnist-conv/model.onnx")
# model1 = ModelWrapper(raw_m)
# model1 = ModelWrapper("resnet50_w1a2_exported.onnx")
model1 = ModelWrapper("tinyyolo-20210831_updated.onnx")
# the input for model1 comes from a uint8 vector so we set the finn datatype
# of the input tensor to DataType["UINT8"] to verify that the datatypes are
# correctly preserved in the transformed model
model1.set_tensor_datatype(model1.graph.input[0].name, DataType["UINT8"])
model1 = model1.transform(InferShapes())
model1 = model1.transform(GiveUniqueNodeNames())
model1 = model1.transform(GiveReadableTensorNames())
# model1.save("m1_chkp.onnx")
print(model1.graph.input)
print(model1.graph.output)

# set up post model
# shape = [1, 3, 512, 512]
shape = model1.get_tensor_shape(model1.graph.input[0].name)
inp = helper.make_tensor_value_info("inp", TensorProto.FLOAT, shape)
a0 = helper.make_tensor_value_info("a0", TensorProto.FLOAT, [])
a1 = helper.make_tensor_value_info("a1", TensorProto.FLOAT, [])
outp = helper.make_tensor_value_info("outp", TensorProto.FLOAT, shape)

mul_node = helper.make_node("Mul", ["inp", "a0"], ["mul_out"])
div_node = helper.make_node("Div", ["inp", "a1"], ["outp"])

graph = helper.make_graph(
    nodes=[div_node],
    name="model2-graph",
    inputs=[inp],
    outputs=[outp],
    value_info=[a0, a1],
)

model2 = helper.make_model(graph, producer_name="model2")
model2 = ModelWrapper(model2)
# initialize model2
a0_value = np.random.uniform(low=0, high=1, size=(1)).astype(np.float32)
model2.set_initializer("a0", a0_value)
a1_value = np.array(255, dtype = np.float32)
model2.set_initializer("a1", a1_value)

model2 = model2.transform(InferShapes())
model2 = model2.transform(InferDataTypes())
model2 = model2.transform(InferDataLayouts())
model2 = model2.transform(GiveUniqueNodeNames())
model2 = model2.transform(GiveReadableTensorNames())
print(model2.graph.input)
print(model2.graph.output)


model1.save('m1.onnx')
model2.save('m2.onnx')

# merge models
model_transformed = model1.transform(MergeONNXModels(model2))

model_transformed.transform(GiveUniqueNodeNames())
model_transformed.save("mt.onnx")
print("Complete")


[name: "global_in"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 512
      }
      dim {
        dim_value: 512
      }
    }
  }
}
]
[name: "global_out"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3840
      }
      dim {
        dim_value: 6
      }
    }
  }
}
]
[name: "global_in"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 512
      }
      dim {
        dim_value: 512
      }
    }
  }
}
]
[name: "global_out"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 512
      }
      dim {
        dim_value: 512
      }
    }
  }
}
]


/workspace/qonnx/src/qonnx/transformation/infer_data_layouts.py:119: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


RuntimeError: Input 1 is out of bounds.